In [1]:
import sys
sys.path.append("../")

In [2]:
from __future__ import division
from full_prod_DRA import *
from buchi import buchi_from_ltl
import numpy as np
from env_sensing_error import *
import scipy
# from plot_path_for_prod import *
from graphviz import Source
from qnetwork import *
from utils import *
import matplotlib.pyplot as plt
# from Plot_Path import *
import tensorflow as tf
from dra_planning import *
import time

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def train(sess, env, qnet, prod_planner):
    
    global EXPLORATION_RATE
    global GUIDE_RATE
  
    summary_ops, summary_vars = build_summaries()
    if not RESTORE:
        sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    
    qnet.update_target()
    
    replay_buffer = ReplayBuffer(BUFFER_SIZE, RANDOM_SEED)
    
    while len(prod_planner.opt_path) == 0:
        env.step(np.random.randint(0,qnet.action_dim))
        prod_planner.update_wfts_ap()
        env.update_dynamic_rabin()
        prod_planner.get_global_opt()
    print "Global Solution Found"
    prod_planner.get_opt_rabin()
    
    saved_dra_planners = {}
    
    for num_epi in range(MAX_EPISODES):
        
#         print "Epi: ", num_epi

        s = env.reset()
        s = list(np.unravel_index(s, env.shape))
#         prod_planner.replace_region_list()
        prod_planner.update_wfts_ap()

        ep_reward = 0
        ep_ave_max_q = 0
        
        reward_list = []
        
        train_time = 0
        batch_time = 0
        gym_time = 0
        guide_time = 0

        for j in range(MAX_EPISODE_LEN):
            
#             print "Step: ", j
            
            gym_start = time.time()

            rand_num = np.random.rand(1)
    
            if rand_num <= EXPLORATION_RATE:
                a = np.random.randint(0,qnet.action_dim)
                s2, r, terminal, info = env.step(a)
                
            elif rand_num <= GUIDE_RATE+EXPLORATION_RATE and rand_num > EXPLORATION_RATE:
#                 print "GUIDE"
                guide_start = time.time()
                
                if rand_num > EXPLORATION_RATE + 0.9*GUIDE_RATE:
                # Only update global plan with 0.2 prob for efficiency
                    env.update_dynamic_rabin()
                    prod_planner.get_global_opt()
                    prod_planner.get_opt_rabin()
                    
                if len(prod_planner.opt_path) > 0:
#                     print "S: ", s
#                     print "last a: ", a
                    new_ltl = prod_planner.get_next_ltl(s[-1])
                    
#                     if new_ltl in saved_dra_planners.keys():
                    guide_path = prod_planner.get_local_opt(s[:-1], new_ltl)
                    if guide_path != None:
#                         print "GUIDE"
                        a = convert_path_to_action(guide_path)
                    else:
                        a = np.random.randint(0,qnet.action_dim)
#                     else:
#                         guide_path = prod_planner.get_local_opt(s[:-1], new_ltl)
#                         saved_dra_planners[new_ltl] = prod_planner.dra_full_prod
#                         if guide_path != None:
#                             a = convert_path_to_action(guide_path)
#                         else:
#                             a = np.random.randint(0,qnet.action_dim)
                        
                else:
                    a = np.random.randint(0,qnet.action_dim)
                
                s2, r, terminal, info = env.step(a)
                
                guide_time += time.time() - guide_start
                
            else:
                a = np.argmax(qnet.predict_q(np.reshape(s, (1, qnet.state_dim))))
                s2, r, terminal, info = env.step(a)

            prod_planner.update_wfts_ap()
                
            gym_time += time.time() - gym_start
            
            batch_start = time.time()
            
            s2 = list(np.unravel_index(s2, env.shape))

            replay_buffer.add(np.reshape(s, (qnet.state_dim,)), np.reshape(a, (1,)), r,
                              terminal, np.reshape(s2, (qnet.state_dim,)))
            batch_time += time.time() - batch_start

            # Keep adding experience to the memory until
            # there are at least minibatch size samples
            if replay_buffer.size() > MINIBATCH_SIZE:
                
                batch_start = time.time()
                s_batch, a_batch, r_batch, t_batch, s2_batch = replay_buffer.sample_batch(MINIBATCH_SIZE)
#                 print "sbatch: ", s_batch
                # Calculate targets
                target_q = qnet.predect_target(s2_batch)

                y_i = []
                for k in range(MINIBATCH_SIZE):
                    if t_batch[k]:
                        y_i.append(r_batch[k])
                    else:
                        y_i.append(r_batch[k] + GAMMA * np.amax(target_q[k]))
                        
                batch_time += time.time() - batch_start

                # Update the critic given the targets
                train_start = time.time()
                predicted_q_value, _ = qnet.train(s_batch, a_batch, np.reshape(y_i, (MINIBATCH_SIZE, 1)), num_epi)

                ep_ave_max_q += np.amax(predicted_q_value)
                
                # Update target networks
                qnet.update_target()

                train_time += time.time() - train_start

            s = s2
            ep_reward += r

            if terminal or j == MAX_EPISODE_LEN-1:
                
                if EXPLORATION_RATE > 0.1 and terminal:
                    EXPLORATION_RATE = EXPLORATION_RATE*0.992
                if GUIDE_RATE > 0.1 and terminal:    
                    GUIDE_RATE = GUIDE_RATE*0.992
                    
                reward_list += [ep_reward]
                
                if np.average(reward_list[-10:]) > LR_DECAY_TRUNCATION:
                    qnet.decay_learning_rate(0.992)

                print('| Reward: {:d} | Episode: {:d} | Qmax: {:.4f} | Exploration: {:.6f} | Step: {:d} | LR: {:.8f}'.format(int(ep_reward), \
                        num_epi, (ep_ave_max_q / float(j)), EXPLORATION_RATE, j, qnet.get_learning_rate()))
                
                f = open("stats/" + file_appendix + "_stats.txt", "ab")
                f.write("| Reward: " + str(int(ep_reward)) 
                        +" | Episode: " + str(num_epi) 
                        + " | Qmax: " + str(ep_ave_max_q / float(j)) 
                        + " | Exploration: " + str(EXPLORATION_RATE)
                        + " | Step: " + str(j)
                        + " | LR:" + str(qnet.get_learning_rate()) + "\n")
                f.close()
                
                f = open("stats/" + file_appendix + "_stats_time.txt", "ab")
                f.write(" | Episode: " + str(num_epi) 
                        + " | Train: " + str(train_time) 
                        + " | Gym: " + str(gym_time)
                        + " | Batch: " + str(batch_time) 
                        + " | Guide: " + str(guide_time)
                        + "\n")
                f.close()
                
                break
                

In [4]:
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B"
# LTL = "[] (p1 -> !(X p1) U (p2 || p3) ) && []<>p1"
# LTL = "T && []<>A && []<>B"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C && []!D"
LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C"
# LTL = "<>(A && <>(B && <> T))"
# LTL = "<>(A && <>B) && <>[]T && []!C"
# LTL = "<>(A && <>T) && []!C"
# LTL = "<>(A && <>(B && <>T)) && []<>(A||T) && []<>B && []!C"
# LTL = "<>(A && <>(B && <>T)) && []!C"
# LTL = "<>(A && <>D) && <>(B && <>E) && []<>T && []<>(D || E) && []!C"

LEARNING_RATE = 0.0001
GAMMA = 0.99
# GAMMA = 0.7
TAU = 0.001
BUFFER_SIZE = 10**6
MINIBATCH_SIZE = 64
RANDOM_SEED = 438
MAX_EPISODES = 50000
MAX_EPISODE_LEN = 1000
file_appendix = "Guide_Planning_2_" + time.ctime()[4:16].replace("  ","").replace(" ","_").replace(":","-") + "_large_" + LTL
# file_appendix = "Feb8_03-47"
SUMMARY_DIR = './results/tf_ddqn_' + file_appendix
SAVE_DIR = "./saved_model/" + file_appendix + "/ddqn.ckpt"
EXPLORATION_RATE = 0.2
GUIDE_RATE = 0.3
LR_DECAY_TRUNCATION = -1500
RESTORE = 0
if sys.platform == "darwin":
    DEVICE = "/device:CPU:0"
else:
    DEVICE = "/device:GPU:0"

In [5]:
env = CurrentWorld(LTL)
prod_planner = Prod_Planning(env, LTL)
# with open("my.dot", "r") as dotfile:
#     text = dotfile.read()
# Source(text)

In [ ]:
config=tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
       
    state_dim = 3
    action_dim = 5
    
    if RESTORE:
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)
        Qnet.saver.restore(sess, RESTORE_PATH)
        train(sess, env, Qnet, prod_planner)
        
    else:
        np.random.seed(RANDOM_SEED)
        tf.set_random_seed(RANDOM_SEED)
        env.seed(RANDOM_SEED)
    
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)

        train(sess, env, Qnet, prod_planner)

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Global Solution Found
DDQN Saved
| Reward: -5662 | Episode: 0 | Qmax: 15.1060 | Exploration: 0.198400 | Step: 533 | LR: 0.00010000
| Reward: -8022 | Episode: 1 | Qmax: 13.5931 | Exploration: 0.196813 | Step: 868 | LR: 0.00010000
| Reward: -11890 | Episode: 2 | Qmax: 3.8628 | Exploration: 0.196813 | Step: 999 | LR: 0.00010000
| Reward: -12250 | Episode: 3 | Qmax: -2.8374 | Exploration: 0.196813 | Step: 999 | LR: 0.00010000
| Reward: -13960 | Episode: 4 | Qmax: -5.8659 | Exploration: 0.196813 | Step: 999 | LR: 0.00010000
| Reward: -12700 | Episode: 5 | Qmax: -7.3748 | Exploration: 0.196813 | Step: 999 | LR: 0.00010000
| Reward: -11440 | Episode: 6 | Qmax: -10.1762 | Exploration: 0.196813 | Step: 999 | LR: 0.00010000
| Reward: -11890 | Episode: 7 | Qmax: -12.4108 | Exploration: 0.196813 | Step: 999 | LR: 0.00010000
| Reward: -11890 | Episode: 8 | Qmax: -16.2202 | Exploration: 0.196813 | Step: 999 | LR: 0.00010000
| Reward: -11080 | Episode: 9 | Qmax: -19.4035 | Exploration: 0.196813 | Ste

| Reward: -10630 | Episode: 81 | Qmax: -5.9873 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -10360 | Episode: 82 | Qmax: -8.3864 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11080 | Episode: 83 | Qmax: -1.6518 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -10540 | Episode: 84 | Qmax: -0.8010 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -7849 | Episode: 85 | Qmax: 6.4146 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -10990 | Episode: 86 | Qmax: 24.5246 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -10900 | Episode: 87 | Qmax: 24.1861 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -10720 | Episode: 88 | Qmax: 22.1765 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11620 | Episode: 89 | Qmax: 25.9049 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11098 | Episode: 90 | Qmax: 28.3338 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| 

| Reward: -11260 | Episode: 162 | Qmax: 192.1265 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -8641 | Episode: 163 | Qmax: 214.7551 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11710 | Episode: 164 | Qmax: 224.1355 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11440 | Episode: 165 | Qmax: 226.9422 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -10270 | Episode: 166 | Qmax: 217.8436 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -8776 | Episode: 167 | Qmax: 224.2556 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11170 | Episode: 168 | Qmax: 247.3792 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11530 | Episode: 169 | Qmax: 226.8278 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -11440 | Episode: 170 | Qmax: 241.6862 | Exploration: 0.195238 | Step: 999 | LR: 0.00010000
| Reward: -10810 | Episode: 171 | Qmax: 233.5125 | Exploration: 0.195238 | Step: 999

In [ ]:
env.reset()

In [ ]:
env.s

In [ ]:
np.where(np.array(prod_planner.opt_rabin) == 1)[0][0]

In [ ]:
np.unravel_index(env.s,(10,10,5))

In [ ]:
np.unravel_index(env.s,(10,10,5))

In [ ]:
ns,r,_,_ = env.step(1)
print np.unravel_index(ns,(10,10,5)),r

In [ ]:
prod_planner.get_global_opt()

In [ ]:
env.ap_dict

In [ ]:
env.last_ap_dict

In [ ]:
env.last_dynamic_coord_dict[(2,2)]

In [ ]:
env.rabin.coord_dict[(2,2)]

In [ ]:
env.dynamic_coord_dict[(2,2)]

In [ ]:
env.rabin.possible_states(1)

In [ ]:
[env.rabin.graph[str(1)][str(0)][k]["label"] for k in range(len(env.rabin.graph[str(1)][str(0)]))]

In [ ]:
env.rabin.check_ap("C", u' !A&!T&C')

In [ ]:
env.rabin.graph["0"]["0"]

In [ ]:
env._calculate_transition_prob((4, 3, 1), [0,1])

In [ ]:
np.unravel_index(221,(10,10,5))

In [ ]:
env.shape

In [ ]:
current = (4,3,1)
delta=[0,1]
delta_list = [[-1, 0], [1, 0], [0, -1], [0, 1], [0, 0]]

In [ ]:
new_position_candidates = [np.array(current[:2]) + np.array(delta)]
new_position_candidates += [np.array(current[:2]) + np.array(i) for i in delta_list if i != delta]

In [ ]:
print new_positions

In [ ]:
new_positions = [env._limit_coordinates(i).astype(int) for i in new_position_candidates]

In [ ]:
next_rabin_state = [env.rabin.next_state(current, tuple(i)) for i in new_positions]

In [ ]:
env.rabin.next_state((4,3,1),(4,4))

In [ ]:
env.rabin.coord_dict[(2,7)]

In [ ]:
env.rabin.next_state((4,3,1),[0,1])

In [ ]:
env.rabin.deadlock

In [ ]:
np.where(np.array(prod_planner.opt_rabin) == 8)[0]

In [ ]:
env = CurrentWorld(LTL.lower())

In [ ]:
prod_planner = Prod_Planning(env, LTL.lower())

In [ ]:
prod_planner.update_wfts_ap()

In [ ]:
env.dynamic_coord_dict[(5,5)]

In [ ]:
env.last_dynamic_coord_dict[(5,5)]

In [ ]:
prod_planner.region_list[np.ravel_multi_index((5,5),(10,10))].ap

In [ ]:
env.ap_dict

In [ ]:
env.last_ap_dict

In [ ]:
env.step(1)